In [24]:
from datasets import load_dataset

In [25]:
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns(["idx"])
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

In [26]:
from sentence_transformers import SentenceTransformer

In [27]:
embedding_model = SentenceTransformer('bert-base-uncased')

In [28]:
from sentence_transformers import losses
train_loss = losses.SoftmaxLoss(
    model=embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3
)

In [29]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts['sentence1'],
    sentences2=val_sts['sentence2'],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine"
)

In [30]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
args = SentenceTransformerTrainingArguments(
    output_dir="base_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100
    )

In [31]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator,
)
# trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [32]:

from datasets import load_dataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
train_dataset = load_dataset(
    "glue", "stsb", split="train"
).select(range(5_000))
train_dataset = train_dataset.remove_columns(["idx"])

val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts['sentence1'],
    sentences2=val_sts['sentence2'],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine"
)

In [33]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

In [35]:
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
train_loss = losses.MultipleNegativesRankingLoss(
    model=embedding_model
)

args = SentenceTransformerTrainingArguments(
    output_dir="stsb_miner_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator,
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,0.488300


TrainOutput(global_step=157, training_loss=0.43808317488166176, metrics={'train_runtime': 537.4768, 'train_samples_per_second': 9.303, 'train_steps_per_second': 0.292, 'total_flos': 0.0, 'train_loss': 0.43808317488166176, 'epoch': 1.0})

In [37]:
evaluator(embedding_model)

{'pearson_cosine': 0.7889591731374033, 'spearman_cosine': 0.8001859251710514}

In [1]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
from sentence_transformers import InputExample
from sentence_transformers.datasets import NoDuplicatesDataLoader

In [2]:
dataset = load_dataset("glue", "mnli", split="train").select(range(1000))
mapping = {2: 0, 1:0, 0:1}
gold_examples = [InputExample(texts=[row['premise'], row['hypothesis']], label=mapping[row['label']]) for row in tqdm(dataset)]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
100%|██████████| 1000/1000 [00:00<00:00, 10569.95it/s]


In [3]:
gold_dataloader = NoDuplicatesDataLoader(
    gold_examples,
    batch_size=32
)
gold =pd.DataFrame({
    "sentence1": dataset["premise"],
    "sentence2": dataset["hypothesis"],
    "label": [mapping[label] for label in dataset["label"]]
})

In [4]:
from sentence_transformers.cross_encoder import CrossEncoder
cross_encoder = CrossEncoder('bert-base-uncased', num_labels=2)
cross_encoder.fit(
    train_dataloader=gold_dataloader,
    epochs=1,
    show_progress_bar=True,
    warmup_steps=100,
    use_amp=True
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: h-emamgholizadeh (h-emamgholizadeh-free-university-of-bozen-bolzano) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


: 

: 

: 

In [ ]:
silver = load_dataset("glue", "mnli", split="validation").select(range(2000))
pairs = list(zip(silver["premise"], silver["hypothesis"]))

In [ ]:
import numpy as np
outputs = cross_encoder.predict(pairs, apply_softmax=True, show_progress_bar=True)
silveer_df = pd.DataFrame({
    "sentence1": silver["premise"],
    "sentence2": silver["hypothesis"],
    "label": np.argmax(outputs, axis=1)
})

In [ ]:
data = pd.concat([gold, silveer_df], ignore_index=True, axis=0)
data.drop_duplicates(subset=["sentence1", "sentence2"], inplace=True, keep='first')

In [ ]:
train_dataset = Dataset.from_pandas(data, PreserveIndex=False)

In [ ]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# Define model
embedding_model = SentenceTransformer("bert-base-uncased")

# Loss function
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# Define the training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="augmented_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
)

# Train model
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator,
)
trainer.train()